In [3]:
import pickle

# Load the pickle file
with open(f"../scenario-properties-collision.pkl", "rb") as f:
    scenario_properties = pickle.load(f)


/opt/anaconda3/envs/opus-env/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [6]:
for i in scenario_properties.collision_pairs:
    print(i.eqs.shape)

(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(36, 17)
(